In [54]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff  # 圖形工廠
from plotly.subplots import make_subplots  # 繪製子圖
import numpy as np
from Upload_git import upload
from os import system
import time
from time import ctime, sleep
from Upload_git import upload
import schedule
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from statsmodels.regression.rolling import RollingOLS, RollingWLS
from scipy import stats
import revenue_function as fn
import pickle

In [4]:
def get_yoy(data):
    data = data.sort_values('Year')
    data['yoy'] = round(data.rev.diff()/data.rev.shift(1), 4)
    return data

def get_st_mom_yoy(data):
    data['Year'] =  data.rev_period.astype(str).str[0:4]
    data['month'] = data.rev_period.astype(str).str[5:7]
    data['mom'] = round(data.rev.diff()/data.rev.shift(1), 4)
    yoy = pd.DataFrame()
    yoy = data.groupby('month').apply(get_yoy).reset_index(drop=True, inplace=False)
    return yoy#[yoy.Year >= '2016']
def get_ratio(data):
    ttl_rev = data.rev.sum()/len(data)*12
    data['ratio'] = round(data.rev/ttl_rev, 4)
    return data


def get_avg_ratio(data):
    if len(data) >= 5:
        weight = [1, 1, 1, 3, 5]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) == 4:
        weight = [1, 1, 2, 3]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) == 3:
        weight = [1, 1, 2]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) <= 2:
        roll_num = len(data)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(data.ratio.mean(), 4)

    
def predict(data):
    a = data.groupby('Year', as_index=False).apply(get_ratio)
#     a.ratio.tail(1).values[0] = round(a.rev.tail(1)/a.rev.rolling(12).sum().tail(1), 4)
    ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
    predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
    predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
    predict['predict_month_rev'] = round(round(predict['predict_annul_rev'].mean(), 2) * predict.wt_avg_ratio)
    predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
    predict['rev'] = predict['predict_month_rev']
    predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
    predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
    predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
    predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
    predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
    return predict.tail(12)


weight = [1, 1, 1, 3, 5]


def get_ratio(data):
    data['ratio'] = round(data.rev/data.rev.sum(), 4)
    return data


def get_avg_ratio(data):
    roll_num = len(weight)
    data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
    return round(np.average(data.ratio, weights = weight), 4)

def predict(data):
    try:
        a = data.groupby('Year', as_index=False).apply(get_ratio)
        ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
        predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
        predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
        predict['predict_month_rev'] = round(round(predict['predict_annul_rev'].mean(), 2) * predict.wt_avg_ratio)
        predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
        predict['rev'] = predict['predict_month_rev']
        predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
        predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
        predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
        predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
        predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
        return predict.tail(12)
    except TypeError:
        pass


def growth(data):
    data['yoy年成長大於10'] = np.where(data.yoy >= 10, 1, 0)
    data['yoy成長'] = np.where(data.yoy > data.yoy.shift(1), 1, 0)
    data['yoy衰退'] = np.where(data.yoy <= data.yoy.shift(1), 1, 0)
    
    test = data.loc[:, ['yoy年成長大於10']]
    data['yoy近五月大於10'] = test.rolling(5).sum()
    data['yoy連續大於10'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy成長']]
    data['yoy近五月成長'] = test.rolling(5).sum()
    data['yoy連續成長'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy衰退']]
    data['yoy近五月衰退'] = test.rolling(5).sum()
    data['yoy連續衰退'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    return data


# a.groupby('st_code', as_index=False).apply(predict).reset_index(drop=True).head(30)

In [56]:
"""
connect to db
"""
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)
db = pd.read_sql('tej_revenue', engine).drop('index', axis=1)

revenue = pd.read_sql('industry', engine)
new_industry_name = revenue['TSE新產業名.1'].dropna()
minor_industry_name = revenue['TEJ子產業名.1'].dropna()
db['rev'] = round(db['rev']/1000, 2)
stock_info = revenue.iloc[:, 1:4].rename({'TSE新產業名':'new_industry_name',
                                          'TEJ子產業名':'minor_industry_name'}, axis=1)
stock_info['st_code'] = stock_info['公司簡稱'].str[0:4]
stock_info['st_name'] = stock_info['公司簡稱'].str[5:]
stock_info = stock_info.drop('公司簡稱', axis=1)

# profit = pd.read_excel('2015income_stat.xlsx')
# profit['營業收入淨額'] = round(profit['營業收入淨額']/1000, 1)
# profit['營業成本'] = round(profit['營業成本']/1000, 1)
# profit['營業毛利'] = round(profit['營業毛利']/1000, 1)
# profit['營業費用'] = round(profit['營業費用']/1000, 1)
# profit['其他收益及費損淨額'] = round(profit['其他收益及費損淨額']/1000, 1)
# profit['營業利益'] = round(profit['營業利益']/1000, 1)
# profit['營業外收入及支出'] = round(profit['營業外收入及支出']/1000, 1)
# profit['稅前淨利'] = round(profit['稅前淨利']/1000, 1)
# profit['所得稅費用'] = round(profit['所得稅費用'], 1)
# profit['其他綜合損益－OCI'] = round(profit['其他綜合損益－OCI']/1000, 1)
# profit['所得稅費用'] = round(profit['所得稅費用']/1000, 1)
# profit['稀釋稅後淨利'] = round(profit['稀釋稅後淨利']/1000, 1)
# profit['歸屬非控制權益淨利（損）'] = round(profit['歸屬非控制權益淨利（損）']/1000, 1)
# profit['稅後淨利est'] = profit['營業收入淨額'] - profit['營業成本'] - profit['營業費用'] + profit['其他收益及費損淨額']+ profit['營業外收入及支出'] - profit['所得稅費用'] - profit['歸屬非控制權益淨利（損）']
# profit['eps_est'] = round(profit['稅後淨利est']/profit['加權平均股數']*1000, 2)
# profit = profit.loc[:, ['代號', '名稱', '年/月', '加權平均股數', '營業收入淨額', '營業成本', '營業毛利', '營業費用',
#                 '其他收益及費損淨額', '營業利益', '營業外收入及支出', '稅前淨利', '所得稅費用', 
#                 '歸屬非控制權益淨利（損）', '稅後淨利est', '每股盈餘']]
# profit = profit[(profit['營業成本'].isna() == False) & (profit['營業毛利'].isna() == False) & (profit['代號'] <= '9999')]

2023-03-08 23:08:33,220 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-08 23:08:33,221 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:08:33,646 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-08 23:08:33,648 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:08:34,072 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-08 23:08:34,073 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:08:34,497 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-08 23:08:34,497 INFO sqlalchemy.engine.Engine [generated in 0.00094s] {'name': 'tej_revenue'}
2023-03-08 23:08:35,128 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-03-08 23:08:35,129 INFO sqlalchemy.engine.Engine [generated i

2023-03-08 23:08:37,509 INFO sqlalchemy.engine.Engine [generated in 0.00061s] {'table_oid': 16512}
2023-03-08 23:08:37,720 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2023-03-08 23:08:37,720 INFO sqlalchemy.engine.Engine [generated in 0.00061s] {'table_oid': 16512}
2023-03-08 23:08:38,140 INFO sqlalchemy.engine.Engine SELECT tej_revenue.index, tej_revenue.st_code, tej_revenue.st_name, tej_revenue.rev_period, tej_revenue.declaration_date, tej_revenue.rev, tej_revenue.declaration_year, tej_revenue.declaration_month, tej_revenue.new_industry_name, tej_revenue.minor_industry_name 
FROM tej_revenue
2023-03-08 23:08:38,141 INFO sqlalchemy.engine.Engine [generated in 0.00069s] {}
2023-03-08 23:09:03,758 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_

2023-03-08 23:09:06,514 INFO sqlalchemy.engine.Engine [cached since 29.22s ago] {'table_oid': 16461}
2023-03-08 23:09:06,724 INFO sqlalchemy.engine.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'c'
        
2023-03-08 23:09:06,725 INFO sqlalchemy.engine.Engine [cached since 29.22s ago] {'table_oid': 16461}
2023-03-08 23:09:06,934 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2023-03-08 23:09:06,935 INFO sqlalchemy.engine.Engine [cached since 29.22s ago] {'table_oid': 16461}
2023-03-08 23:09:07,367 INFO sqlalchemy.engine.Engine SELECT in

In [4]:
st_future = pd.read_sql('個股期歷史資料（近月及近二月）', engine)

2023-03-08 15:37:30,951 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-08 15:37:30,953 INFO sqlalchemy.engine.Engine [cached since 60.46s ago] {'name': '個股期歷史資料（近月及近二月）'}
2023-03-08 15:37:31,828 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-03-08 15:37:31,828 INFO sqlalchemy.engine.Engine [cached since 60.42s ago] {'schema': 'public'}
2023-03-08 15:37:32,444 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        
2023-03-08 15:37:32,445 INFO sqlalchemy.engine.En

2023-03-08 15:37:35,236 INFO sqlalchemy.engine.Engine [cached since 60.44s ago] {'table_oid': 16661}
2023-03-08 15:37:35,826 INFO sqlalchemy.engine.Engine SELECT "個股期歷史資料（近月及近二月）".index, "個股期歷史資料（近月及近二月）"."代號", "個股期歷史資料（近月及近二月）"."名稱", "個股期歷史資料（近月及近二月）"."日期", "個股期歷史資料（近月及近二月）"."到期月", "個股期歷史資料（近月及近二月）"."開盤價", "個股期歷史資料（近月及近二月）"."最高價", "個股期歷史資料（近月及近二月）"."最低價", "個股期歷史資料（近月及近二月）"."收盤價", "個股期歷史資料（近月及近二月）"."每日結算價", "個股期歷史資料（近月及近二月）"."報酬率", "個股期歷史資料（近月及近二月）"."成交張數(量)", "個股期歷史資料（近月及近二月）"."未平倉合約數", "個股期歷史資料（近月及近二月）"."基差", "個股期歷史資料（近月及近二月）"."標的證券價格", "個股期歷史資料（近月及近二月）"."最後揭示買價", "個股期歷史資料（近月及近二月）"."最後揭示賣價", "個股期歷史資料（近月及近二月）"."標的證券ROI%%", "個股期歷史資料（近月及近二月）"."標的證券", "個股期歷史資料（近月及近二月）"."成交量(合計)", "個股期歷史資料（近月及近二月）"."未沖銷契約數(合計)" 
FROM "個股期歷史資料（近月及近二月）"
2023-03-08 15:37:35,827 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {}


In [672]:
# st_future = st_future[(st_future.st_code.str.isdigit() == True)]
# st_future = st_future.loc[:, ['代號', '標的證券', '日期', '到期月', '開盤價', '最高價', 
#                               '最低價', '每日結算價', '報酬率', '成交張數(量)',
#                               '成交量(合計)']]
# st_future['近遠月'] = np.where(st_future['代號'].str[-1:] == 'A', '近月', '近兩月')
# st_future['st_code'] = st_future['標的證券'].str[0:4]
# st_future['st_name'] = st_future['標的證券'].str[5:]
# st_future = st_future.sort_values('日期').drop('標的證券', axis=1)
# st_future[st_future.st_code == '2834'].tail(20)
# st_future[st_future['代號']=='ZGTF'] 加權
# st_future = st_future.loc[:, ['st_code', 'st_name', '代號', '日期', '到期月', '開盤價', '最高價',
#                               '最低價', '每日結算價', '報酬率', '成交張數(量)', '成交量(合計)']]
st_future

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost
619385,2301,光寶科,ZFQF,2015/03/09,2015/03,40.1000,40.4214,40.1000,40.200,-1.3497,36,69,近兩月,None,9,2015,03,40.2
619490,3035,智原,ZIPFA,2015/03/09,2015/03,41.9500,43.2000,41.7000,43.000,2.9940,369,676,近月,None,9,2015,03,43.0
619573,3044,健鼎,ZLBF,2015/03/09,2015/03,65.3857,65.6571,64.6000,64.600,-1.6849,118,225,近兩月,None,9,2015,03,64.6
619572,2328,廣宇,ZKWFA,2015/03/09,2015/03,18.4000,18.5000,18.4000,18.400,-1.0753,31,61,近月,None,9,2015,03,18.4
619571,2328,廣宇,ZKWF,2015/03/09,2015/03,18.4214,18.4786,18.4000,18.400,-1.1704,24,61,近兩月,None,9,2015,03,18.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618896,2458,義隆,ZGWF,2023/03/07,2023/03,104.5833,104.5833,104.0000,104.000,-0.3195,19,33,近兩月,2023/02,7,2023,03,13.2
618895,2455,全新,ZGUFA,2023/03/07,2023/03,85.5000,87.0000,83.5000,83.700,-1.8757,272,292,近月,2023/02,7,2023,03,13.2
618982,9904,寶成,ZKLF,2023/03/07,2023/03,34.1000,34.2500,34.1000,34.200,0.2933,24,46,近兩月,2023/02,7,2023,03,13.2
618857,2301,光寶科,ZFQFA,2023/03/07,2023/03,75.1000,75.9000,74.9000,75.400,1.4805,132,172,近月,2023/02,7,2023,03,13.2


In [216]:
def get_yoy(data):
    data = data.sort_values('Year')
    data['yoy'] = round(data.rev.diff()/data.rev.shift(1), 4)
    return data


def get_st_mom_yoy(data):
    data['Year'] =  data.rev_period.astype(str).str[0:4]
    data['month'] = data.rev_period.astype(str).str[5:7]
    data['mom'] = round(data.rev.diff()/data.rev.shift(1), 4)
    yoy = pd.DataFrame()
    yoy = data.groupby('month').apply(get_yoy).reset_index(drop=True, inplace=False)
    return yoy


def get_ratio(data):
    ttl_rev = data.rev.sum()/len(data)*12
    data['ratio'] = round(data.rev/ttl_rev, 4)
    return data


def get_avg_ratio(data):
    if len(data) >= 5:
        weight = [1, 1, 1, 3, 5]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 4:
        weight = [1, 1, 2, 3]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 3:
        weight = [1, 1, 2]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 2:
        roll_num = len(data)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(data.ratio.mean(), 4)
        

def get_predict(data):
    # new_industry_list = ['M1722 生技醫療', 'M2500 建材營造', 'M9900 其他', 'M1200 食品工業']
    # minor_industry_list = ['M25A 建設']
    # if (data.new_industry_name.head(1).isin(new_industry_list).values[0] == False) & (data.new_industry_name.head(1).isin(minor_industry_list).values[0] == False):
    if data.shape[0] >= 10:
        a = data.groupby('Year', as_index=False).apply(get_ratio)
        ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
        predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
        predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
        predict['predict_month_rev'] = round(predict['predict_annul_rev'].tail(12).mean()* predict.wt_avg_ratio)
        predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
        predict['rev'] = predict['predict_month_rev']
        predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
        predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
        predict = predict.groupby('month', as_index=False).apply(fn.get_yoy).sort_values(['Year', 'month'])
        predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
        predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
        return predict.tail(12)
    else:
        print(data.st_code.head(1).values[0], data.shape[0])

In [64]:
data = db.groupby(by='st_code', as_index=False).apply(get_st_mom_yoy).reset_index(drop=True,inplace=False).sort_values('rev_period')


In [139]:
data
# mask = 
rev_future = data[data.st_code.isin(st_future.drop_duplicates().st_code) == True]
rev_future.pivot_table(
    index='rev_period',
    columns='st_name',
    aggfunc='count').sum().min()

13.0

In [897]:
# db.groupby(by='st_code', as_index=False).apply(get_st_mom_yoy).reset_index(drop=True,inplace=False).sort_values('rev_period')
# rev_future
full_data = db[db.st_code.isin(st_future.drop_duplicates().st_code) == True].sort_values(['st_code', 'rev_period']).groupby(by='st_code', as_index=False).apply(get_st_mom_yoy).reset_index(drop=True,inplace=False)

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,month,mom,yoy
0,1101,台泥,2013/01,2013/02/18,9134.46,2013,2,M1100 水泥工業,M11A 水泥製造,2013,01,NaN,NaN
1,1101,台泥,2013/02,2013/03/08,5540.35,2013,3,M1100 水泥工業,M11A 水泥製造,2013,02,-0.3935,NaN
2,1101,台泥,2013/03,2013/04/10,9457.97,2013,4,M1100 水泥工業,M11A 水泥製造,2013,03,0.7071,NaN
3,1101,台泥,2013/04,2013/05/10,9919.27,2013,5,M1100 水泥工業,M11A 水泥製造,2013,04,0.0488,NaN
4,1101,台泥,2013/05,2013/06/10,9543.78,2013,6,M1100 水泥工業,M11A 水泥製造,2013,05,-0.0379,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33177,9945,潤泰新,2022/09,2022/10/11,2627.72,2022,10,M9900 其他,M25A 建設,2022,09,0.1308,0.3292
33178,9945,潤泰新,2022/10,2022/11/10,2360.37,2022,11,M9900 其他,M25A 建設,2022,10,-0.1017,0.1781
33179,9945,潤泰新,2022/11,2022/12/09,2747.26,2022,12,M9900 其他,M25A 建設,2022,11,0.1639,0.4134
33180,9945,潤泰新,2022/12,2023/01/10,5195.96,2023,1,M9900 其他,M25A 建設,2022,12,0.8913,0.6198


In [234]:
def get_wet_reg(data):
    if data.shape[0] < 24:
        return []
    a = get_rolling_predict(data).head(1)
    return [a.st_code.values[0], a.rev_period.values[0], a.rev.values[0], a.mom.values[0], a.yoy.values[0]]


def rolling_predicting(data):
    t = data.rolling(24)  #.iloc[:, :]
    try:
        print('現在進行到', data.st_code.values[0])
        WLS = pd.DataFrame((map(get_wet_reg, t))).set_axis(['st_code', 'rev_period', 'rev_predict24', 'mom_predict24', 'yoy_predict24'], axis=1)
        return WLS.dropna(subset='st_code', axis=0)
    except ValueError:
        pass

# result24 = rev_future.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)
full24 = full_data.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)

現在進行到 1101
現在進行到 1102
現在進行到 1201
現在進行到 1210
現在進行到 1216
現在進行到 1301
現在進行到 1303
現在進行到 1304
現在進行到 1308
現在進行到 1312
現在進行到 1314
現在進行到 1319
現在進行到 1326
現在進行到 1402
現在進行到 1409
現在進行到 1440
現在進行到 1447
現在進行到 1476
現在進行到 1477
現在進行到 1504
現在進行到 1532
現在進行到 1536
現在進行到 1565
現在進行到 1590
現在進行到 1605
現在進行到 1702
現在進行到 1707
現在進行到 1710
現在進行到 1711
現在進行到 1714
現在進行到 1717
現在進行到 1718
現在進行到 1722
現在進行到 1723
現在進行到 1736
現在進行到 1785
現在進行到 1789
現在進行到 1802
現在進行到 1904
現在進行到 1907
現在進行到 1909
現在進行到 2002
現在進行到 2006
現在進行到 2014
現在進行到 2027
現在進行到 2049
現在進行到 2059
現在進行到 2062
現在進行到 2101
現在進行到 2105
現在進行到 2106
現在進行到 2107
現在進行到 2201
現在進行到 2231
現在進行到 2301
現在進行到 2303
現在進行到 2308
現在進行到 2313
現在進行到 2317
現在進行到 2323
現在進行到 2324
現在進行到 2327
現在進行到 2328
現在進行到 2330
現在進行到 2331
現在進行到 2332
現在進行到 2337
現在進行到 2338
現在進行到 2344
現在進行到 2345
現在進行到 2347
現在進行到 2352
現在進行到 2353
現在進行到 2354
現在進行到 2355
現在進行到 2356
現在進行到 2357
現在進行到 2360
現在進行到 2362
現在進行到 2371
現在進行到 2374
現在進行到 2376
現在進行到 2377
現在進行到 2379
現在進行到 2382
現在進行到 2383
現在進行到 2385
現在進行到 2388
現在進行到 2392
現在進行到 2393
現在進行到 2401

In [233]:
i = 0
def get_wet_reg(data):
    if data.shape[0] < 36:
        return []
    a = get_rolling_predict(data).head(1)
    return [a.st_code.values[0], a.rev_period.values[0], a.rev.values[0], a.mom.values[0], a.yoy.values[0]]


def rolling_predicting(data):
    i += 1
    if i % 100 == 99:
        print('現在進行到', data.st_code.values[0])
    t = data.rolling(36)  #.iloc[:, :]
    try:
        WLS = pd.DataFrame((map(get_wet_reg, t))).set_axis(['st_code', 'rev_period', 'rev_predict36', 'mom_predict36', 'yoy_predict36'], axis=1)
        return WLS.dropna(subset='st_code', axis=0)
    except ValueError:
        pass

# result36 = rev_future.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)
full36 = full_data.groupby(by='st_code', as_index=False).parallel_apply(rolling_predicting).reset_index(drop=True,inplace=False)

現在進行到 1101
現在進行到 1102
現在進行到 1201
現在進行到 1210
現在進行到 1216
現在進行到 1301
現在進行到 1303
現在進行到 1304
現在進行到 1308
現在進行到 1312
現在進行到 1314
現在進行到 1319
現在進行到 1326
現在進行到 1402
現在進行到 1409
現在進行到 1440
現在進行到 1447
現在進行到 1476
現在進行到 1477
現在進行到 1504
現在進行到 1532
現在進行到 1536
現在進行到 1565
現在進行到 1590
現在進行到 1605
現在進行到 1702
現在進行到 1707
現在進行到 1710
現在進行到 1711
現在進行到 1714
現在進行到 1717
現在進行到 1718
現在進行到 1722
現在進行到 1723
現在進行到 1736
現在進行到 1785
現在進行到 1789
現在進行到 1802
現在進行到 1904
現在進行到 1907
現在進行到 1909
現在進行到 2002
現在進行到 2006
現在進行到 2014
現在進行到 2027
現在進行到 2049
現在進行到 2059
現在進行到 2062
現在進行到 2101
現在進行到 2105
現在進行到 2106
現在進行到 2107
現在進行到 2201
現在進行到 2231
現在進行到 2301
現在進行到 2303
現在進行到 2308
現在進行到 2313
現在進行到 2317
現在進行到 2323
現在進行到 2324
現在進行到 2327
現在進行到 2328
現在進行到 2330
現在進行到 2331
現在進行到 2332
現在進行到 2337
現在進行到 2338
現在進行到 2344
現在進行到 2345
現在進行到 2347
現在進行到 2352
現在進行到 2353
現在進行到 2354
現在進行到 2355
現在進行到 2356
現在進行到 2357
現在進行到 2360
現在進行到 2362
現在進行到 2371
現在進行到 2374
現在進行到 2376
現在進行到 2377
現在進行到 2379
現在進行到 2382
現在進行到 2383
現在進行到 2385
現在進行到 2388
現在進行到 2392
現在進行到 2393
現在進行到 2401

In [235]:
def get_wet_reg(data):
    if data.shape[0] < 48:
        return []
    a = get_rolling_predict(data).head(1)
    return [a.st_code.values[0], a.rev_period.values[0], a.rev.values[0], a.mom.values[0], a.yoy.values[0]]


def rolling_predicting(data):
    t = data.rolling(48)  #.iloc[:, :]
    try:
        print('現在進行到', data.st_code.values[0])
        WLS = pd.DataFrame((map(get_wet_reg, t))).set_axis(['st_code', 'rev_period', 'rev_predict48', 'mom_predict48', 'yoy_predict48'], axis=1)
        return WLS.dropna(subset='st_code', axis=0)
    except ValueError:
        pass

# result48 = rev_future.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)
full48 = full_data.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)

現在進行到 1101
現在進行到 1102
現在進行到 1201
現在進行到 1210
現在進行到 1216
現在進行到 1301
現在進行到 1303
現在進行到 1304
現在進行到 1308
現在進行到 1312
現在進行到 1314
現在進行到 1319
現在進行到 1326
現在進行到 1402
現在進行到 1409
現在進行到 1440
現在進行到 1447
現在進行到 1476
現在進行到 1477
現在進行到 1504
現在進行到 1532
現在進行到 1536
現在進行到 1565
現在進行到 1590
現在進行到 1605
現在進行到 1702
現在進行到 1707
現在進行到 1710
現在進行到 1711
現在進行到 1714
現在進行到 1717
現在進行到 1718
現在進行到 1722
現在進行到 1723
現在進行到 1736
現在進行到 1785
現在進行到 1789
現在進行到 1802
現在進行到 1904
現在進行到 1907
現在進行到 1909
現在進行到 2002
現在進行到 2006
現在進行到 2014
現在進行到 2027
現在進行到 2049
現在進行到 2059
現在進行到 2062
現在進行到 2101
現在進行到 2105
現在進行到 2106
現在進行到 2107
現在進行到 2201
現在進行到 2231
現在進行到 2301
現在進行到 2303
現在進行到 2308
現在進行到 2313
現在進行到 2317
現在進行到 2323
現在進行到 2324
現在進行到 2327
現在進行到 2328
現在進行到 2330
現在進行到 2331
現在進行到 2332
現在進行到 2337
現在進行到 2338
現在進行到 2344
現在進行到 2345
現在進行到 2347
現在進行到 2352
現在進行到 2353
現在進行到 2354
現在進行到 2355
現在進行到 2356
現在進行到 2357
現在進行到 2360
現在進行到 2362
現在進行到 2371
現在進行到 2374
現在進行到 2376
現在進行到 2377
現在進行到 2379
現在進行到 2382
現在進行到 2383
現在進行到 2385
現在進行到 2388
現在進行到 2392
現在進行到 2393
現在進行到 2401

In [236]:
i = 0
def get_wet_reg(data):
    if data.shape[0] < 60:
        return []
    a = get_rolling_predict(data).head(1)
    return [a.st_code.values[0], a.rev_period.values[0], a.rev.values[0], a.mom.values[0], a.yoy.values[0]]


def rolling_predicting(data):
    i += 1
    if i % 100 == 99:
        print('現在進行到', data.st_code.values[0])
        
    t = data.rolling(60)  #.iloc[:, :]
    try:
        print('現在進行到', data.st_code.values[0])
        WLS = pd.DataFrame((map(get_wet_reg, t))).set_axis(['st_code', 'rev_period', 'rev_predict60', 'mom_predict60', 'yoy_predict60'], axis=1)
        return WLS.dropna(subset='st_code', axis=0)
    except ValueError:
        pass

# result60 = rev_future.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)
full60 = full_data.groupby(by='st_code', as_index=False).apply(rolling_predicting).reset_index(drop=True,inplace=False)

現在進行到 1101
現在進行到 1102
現在進行到 1201
現在進行到 1210
現在進行到 1216
現在進行到 1301
現在進行到 1303
現在進行到 1304
現在進行到 1308
現在進行到 1312
現在進行到 1314
現在進行到 1319
現在進行到 1326
現在進行到 1402
現在進行到 1409
現在進行到 1440
現在進行到 1447
現在進行到 1476
現在進行到 1477
現在進行到 1504
現在進行到 1532
現在進行到 1536
現在進行到 1565
現在進行到 1590
現在進行到 1605
現在進行到 1702
現在進行到 1707
現在進行到 1710
現在進行到 1711
現在進行到 1714
現在進行到 1717
現在進行到 1718
現在進行到 1722
現在進行到 1723
現在進行到 1736
現在進行到 1785
現在進行到 1789
現在進行到 1802
現在進行到 1904
現在進行到 1907
現在進行到 1909
現在進行到 2002
現在進行到 2006
現在進行到 2014
現在進行到 2027
現在進行到 2049
現在進行到 2059
現在進行到 2062
現在進行到 2101
現在進行到 2105
現在進行到 2106
現在進行到 2107
現在進行到 2201
現在進行到 2231
現在進行到 2301
現在進行到 2303
現在進行到 2308
現在進行到 2313
現在進行到 2317
現在進行到 2323
現在進行到 2324
現在進行到 2327
現在進行到 2328
現在進行到 2330
現在進行到 2331
現在進行到 2332
現在進行到 2337
現在進行到 2338
現在進行到 2344
現在進行到 2345
現在進行到 2347
現在進行到 2352
現在進行到 2353
現在進行到 2354
現在進行到 2355
現在進行到 2356
現在進行到 2357
現在進行到 2360
現在進行到 2362
現在進行到 2371
現在進行到 2374
現在進行到 2376
現在進行到 2377
現在進行到 2379
現在進行到 2382
現在進行到 2383
現在進行到 2385
現在進行到 2388
現在進行到 2392
現在進行到 2393
現在進行到 2401

# 預估完結果 rolling 完整版

In [785]:
# predict_data = pd.merge(full24.dropna(), full36.dropna(), on=['st_code', 'rev_period'], how='left')
# predict_data = pd.merge(predict_data, full48.dropna(), on=['st_code', 'rev_period'], how='left')
# predict_data = pd.merge(predict_data, full60.dropna(), on=['st_code', 'rev_period'], how='left')

# pd.merge(data.drop(['declaration_year', 'declaration_month', 'new_industry_name', 'minor_industry_name'], 
# axis=1), predict_data[predict_data.st_code == '2412'], on=['st_code', 'rev_period'], how='right').tail(20)
# predict_data[predict_data.st_code == '2412']#.tail(20)

best_predict = pd.merge(data, predict_data.dropna(subset='yoy_predict60'), 
                        on=['st_code', 'rev_period'], how='inner')
# best_predict.to_sql('best_predict', engine)

# def get_error(best_predict):
#     return(round((abs(best_predict.rev - best_predict.rev_predict24)), 4), 
#           round((abs(best_predict.rev - best_predict.rev_predict36)), 4), 
#           round((abs(best_predict.rev - best_predict.rev_predict48)), 4), 
#           round((abs(best_predict.rev - best_predict.rev_predict60)), 4))
#     return ((np.sum(np.absolute(best_predict.rev - best_predict.rev_predict24))/ len(best_predict.rev), 4),
#             (np.sum(np.absolute(best_predict.rev - best_predict.rev_predict36))/ len(best_predict.rev), 4)
#             (np.sum(np.absolute(best_predict.rev - best_predict.rev_predict48))/ len(best_predict.rev), 4)
#             (np.sum(np.absolute(best_predict.rev - best_predict.rev_predict60))/ len(best_predict.rev), 4))


best_predict.groupby('new_industry_name').apply(get_error)

TypeError: 'tuple' object is not callable

# 營收公告後會成長？

In [1048]:
st_future['rev_period'] = st_future['日期'].str[0:7]
st_future['rev_period'] = np.where(st_future.day < 10.0, None, st_future['rev_period'])
st_future['rev_period'] = st_future['rev_period'].fillna(method='pad')

# pd.merge(st_future[st_future['近遠月'] == '近月'], data[data.yoy >= 10], on=['st_code', 'rev_period'], how='left').sort_values(['st_code', '日期']).dropna(subset='st_name_y').head(30)
data['公布月'] = data.declaration_date.str[0:7]
pd.merge(st_future[st_future['近遠月'] == '近月'],
         data[data.yoy >= 10].loc[
             :, ['st_code', '公布月', 'declaration_date', 'new_industry_name', 'minor_industry_name', 
                 'rev', 'mom', 'yoy']].rename({'公布月':'rev_period'}, axis=1),
         on=['st_code', 'rev_period'], how='inner').head(50)

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost,declaration_date,new_industry_name,minor_industry_name,rev,mom,yoy
0,5534,長虹,ZJWFA,2016/02/15,2016/02,42.40,42.40,42.15,42.15,-0.4723,2,4,近月,2016/02,15,2016,02,42.1500,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
1,5534,長虹,ZJWFA,2016/02/16,2016/02,43.65,43.65,43.50,43.65,3.5587,2,5,近月,2016/02,16,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
2,5534,長虹,ZJWFA,2016/02/17,2016/02,43.75,43.75,43.20,43.02,-1.4433,6,11,近月,2016/02,17,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
3,5534,長虹,ZJWFA,2016/02/18,2016/03,43.30,43.50,43.30,43.50,1.1158,8,8,近月,2016/02,18,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
4,5534,長虹,ZJWFA,2016/02/19,2016/03,42.95,42.95,42.75,42.75,-1.7241,4,4,近月,2016/02,19,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
5,5534,長虹,ZJWFA,2016/02/22,2016/03,43.15,43.15,43.15,43.15,0.9357,1,1,近月,2016/02,22,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
6,5534,長虹,ZJWFA,2016/02/23,2016/03,NaN,NaN,NaN,43.10,-0.1159,0,0,近月,2016/02,23,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
7,5534,長虹,ZJWFA,2016/02/24,2016/03,NaN,NaN,NaN,42.90,-0.4640,0,1,近月,2016/02,24,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
8,5534,長虹,ZJWFA,2016/02/25,2016/03,43.00,43.15,43.00,43.15,0.5828,5,6,近月,2016/02,25,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582
9,5534,長虹,ZJWFA,2016/02/26,2016/03,42.95,42.95,42.95,42.95,-0.4635,1,1,近月,2016/02,26,2016,02,30.9000,2016/02/15,M2500 建材營造,M25A 建設,2657.86,3.7836,15.8582


## 分群回測
1. 每月11號左右調整持股，持有一個月

   預估 2022/01，2021/02/01-10 會公告，所以 2022/01/11 買強勢、賣弱勢，持有到 2022/02/11。
   
2. 分四類看報酬率
-10/0/30/30上

In [1147]:
predict_data = pd.merge(data.loc[:, ['st_code', 'rev_period', 'declaration_date']], full60, 
                        on=['st_code', 'rev_period'], how='right').dropna(subset='st_code', axis=0)

# predict_data = predict_data.groupby('st_code', as_index=False).apply(reset_rev_period)
predict_data

,st_code,rev_period,declaration_date,rev_predict60,mom_predict60,yoy_predict60
0,1101,2018/01,2018/02/12,7049.0,-0.358491,0.1496
1,1101,2018/02,2018/03/12,5217.0,-0.457545,-0.1254
2,1101,2018/03,2018/04/10,8460.0,0.758222,-0.0686
3,1101,2018/04,2018/05/10,8343.0,-0.144320,-0.0033
4,1101,2018/05,2018/06/11,8674.0,-0.213130,0.0223
...,...,...,...,...,...,...
16839,9945,2022/10,2022/11/10,2224.0,-0.153639,0.1100
16840,9945,2022/11,2022/12/09,2549.0,0.079915,0.3114
16841,9945,2022/12,2023/01/10,3266.0,0.188821,0.0181
16842,9945,2023/01,2023/02/10,2511.0,-0.516740,0.0059


In [1096]:
def reset_rev_period(data):
    data['rev_period'] = data['rev_period'].shift(-1)
    return data

def get_cluster(data):
#     data['分類'] = pd.qcut(data.yoy_predict60, 8, labels=[1, 2, 3, 4, 5, 6, 7, 8]) #
#     data['分類'] = pd.cut(data.yoy_predict60, bins=[-10000, -0.1, 0, 0.25, 10000000], 
#                         labels=['小於-10', '-10到0', '0到25', '25以上'])#, '25以上'
    data['分類'] = pd.cut(data.yoy_predict60, bins=[-10000, -0.10, 0, 0.10, 10000000], 
                        labels=['小於-10', '-10到0', '0到10', '10以上']) #, '25以上'
    return data


predict_data = predict_data.groupby('rev_period', as_index=False).apply(get_cluster)


st_future['rev_period'] = st_future['日期'].str[0:7]

# def reset_rev_period(data):
#     data['rev_period'] = np.where(data.day < 11.0, None, st_future['rev_period'])
#     data['rev_period'] = data['rev_period'].fillna(method='pad')
#     return data
# st_future = st_future.groupby('st_code', as_index=False).apply(reset_rev_period).reset_index(drop=False)

st_future['rev_period'] = np.where(st_future.day < 11.0, None, st_future['rev_period'])
st_future['rev_period'] = st_future['rev_period'].fillna(method='pad')

test = pd.merge(st_future[(st_future['近遠月'] == '近月')], predict_data, 
                on=['st_code', 'rev_period'], how='right').sort_values(['st_code', 'rev_period', '日期'])

def get_return(data):
    data['return'] = data['每日結算價'].pct_change(periods=1)
    data['cost'] = data.drop_duplicates('rev_period', keep='first')['每日結算價']
    data['cost'] = data['cost'].fillna(method='pad')
    return data


test = test.groupby('st_code', as_index=False).apply(get_return)
all_industry = test.pivot_table(values='return', index='日期', columns='分類', aggfunc='mean', fill_value=0)
# a['bench_mark'] = test.groupby('日期').agg({'return':'mean'})['return']
px.line(all_industry.cumsum())
# go.Scatter(all_industry.cumsum())
test.sort_values(['st_code', '日期']).head(30)

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost,declaration_date,new_industry_name,rev_predict60,mom_predict60,yoy_predict60,分類,return
0,1101,台泥,ZDFFA,2018/01/11,2018/01,38.25,38.25,37.40,37.50,-2.7237,227.0,407.0,近月,2018/01,11.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,NaN
1,1101,台泥,ZDFFA,2018/01/12,2018/01,38.50,38.50,37.45,37.45,-0.1333,97.0,170.0,近月,2018/01,12.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.001333
2,1101,台泥,ZDFFA,2018/01/15,2018/01,37.65,37.85,37.65,37.85,1.0681,106.0,238.0,近月,2018/01,15.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.010681
3,1101,台泥,ZDFFA,2018/01/16,2018/01,37.90,38.10,37.90,38.00,0.3963,36.0,40.0,近月,2018/01,16.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.003963
4,1101,台泥,ZDFFA,2018/01/17,2018/01,38.05,38.55,38.05,38.35,0.9211,270.0,579.0,近月,2018/01,17.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.009211
5,1101,台泥,ZDFFA,2018/01/18,2018/02,38.50,38.65,37.90,37.90,-1.1734,161.0,270.0,近月,2018/01,18.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.011734
6,1101,台泥,ZDFFA,2018/01/19,2018/02,38.00,38.20,37.85,38.00,0.2639,169.0,334.0,近月,2018/01,19.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.002639
7,1101,台泥,ZDFFA,2018/01/22,2018/02,37.95,37.95,37.65,37.75,-0.6579,135.0,247.0,近月,2018/01,22.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.006579
8,1101,台泥,ZDFFA,2018/01/23,2018/02,37.75,37.75,36.90,37.20,-1.4570,251.0,508.0,近月,2018/01,23.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.014570
9,1101,台泥,ZDFFA,2018/01/24,2018/02,37.50,37.50,37.05,37.30,0.2688,637.0,1268.0,近月,2018/01,24.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.002688


In [1102]:
get_summarize(test)

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost,declaration_date,new_industry_name,rev_predict60,mom_predict60,yoy_predict60,分類,return,單次報酬,count,交易次數,勝報酬,敗報酬,勝個數,勝率
0,1101,台泥,ZDFFA,2018/01/11,2018/01,38.25,38.25,37.40,37.50,-2.7237,227.0,407.0,近月,2018/01,11.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,NaN,NaN,1,1,NaN,NaN,NaN,0
1,1101,台泥,ZDFFA,2018/01/12,2018/01,38.50,38.50,37.45,37.45,-0.1333,97.0,170.0,近月,2018/01,12.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.001333,0.0,1,0,0.0,NaN,0.0,0
2,1101,台泥,ZDFFA,2018/01/15,2018/01,37.65,37.85,37.65,37.85,1.0681,106.0,238.0,近月,2018/01,15.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.010681,0.0,1,0,0.0,NaN,0.0,0
3,1101,台泥,ZDFFA,2018/01/16,2018/01,37.90,38.10,37.90,38.00,0.3963,36.0,40.0,近月,2018/01,16.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.003963,0.0,1,0,0.0,NaN,0.0,0
4,1101,台泥,ZDFFA,2018/01/17,2018/01,38.05,38.55,38.05,38.35,0.9211,270.0,579.0,近月,2018/01,17.0,2018,01,37.5,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.009211,0.0,1,0,0.0,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245297,9945,潤泰新,ZKPFA,2023/03/01,2023/03,43.90,43.90,43.20,43.40,-2.2523,87.0,96.0,近月,2023/02,1.0,2023,03,44.1,NaN,NaN,1502.0,-0.101379,0.0112,0到10,-0.022523,0.0,1,0,0.0,NaN,0.0,0
245298,9945,潤泰新,ZKPFA,2023/03/02,2023/03,43.40,43.40,42.95,43.30,-0.2304,72.0,105.0,近月,2023/02,2.0,2023,03,44.1,NaN,NaN,1502.0,-0.101379,0.0112,0到10,-0.002304,0.0,1,0,0.0,NaN,0.0,0
245299,9945,潤泰新,ZKPFA,2023/03/03,2023/03,43.00,43.30,43.00,43.30,0.0000,24.0,24.0,近月,2023/02,3.0,2023,03,44.1,NaN,NaN,1502.0,-0.101379,0.0112,0到10,0.000000,0.0,1,0,0.0,NaN,0.0,0
245300,9945,潤泰新,ZKPFA,2023/03/06,2023/03,43.30,43.55,43.30,43.55,0.5774,24.0,25.0,近月,2023/02,6.0,2023,03,44.1,NaN,NaN,1502.0,-0.101379,0.0112,0到10,0.005774,0.0,1,0,0.0,NaN,0.0,0


In [1119]:
pd.set_option('display.max_columns', None)
predict_data = pd.merge(data.loc[:, ['st_code', 'rev_period', 'declaration_date']], full60, 
                        on=['st_code', 'rev_period'], how='right').dropna(subset='st_code', axis=0)

def get_cluster(data):
    data['分類'] = pd.cut(data.yoy_predict60, bins=[-10000, -0.10, 0, 0.1, 10000000], 
                        labels=['小於-10', '-10到0', '0到10', '10以上']) #, '25以上'
    return data

predict_data = predict_data.groupby('rev_period', as_index=False).apply(get_cluster)
# predict_data = predict_data.dropna(subset='分類')
st_future['rev_period'] = np.where(st_future.day < 11.0, None, st_future['rev_period'])
st_future['rev_period'] = st_future['rev_period'].fillna(method='pad')


test = pd.merge(st_future[(st_future['近遠月'] == '近月')], predict_data,
                on=['st_code', 'rev_period'], how='right').sort_values(['st_code', 'rev_period', '日期'])

test = test.dropna(subset=['每日結算價'])

def get_return(data):
    data['return'] = data['每日結算價'].pct_change(periods=1)
    data['cost'] = data.drop_duplicates('rev_period', keep='first')['每日結算價']
    data['cost'] = data['cost'].fillna(method='pad')#.shift(1)
    return data

test = test.groupby('st_code', as_index=False).apply(get_return)

all_industry = test.pivot_table(values='return', index='日期', columns='分類', aggfunc='mean', fill_value=0)
# # a['bench_mark'] = test.groupby('日期').agg({'return':'mean'})['return']
px.line(all_industry.cumsum(), title='過濾產業的結果')

In [1120]:
get_summarize(test)
# sharp_ratio(all_industry)

分類,小於-10,-10到0,0到10,10以上
交易次數,27.819672,60.639344,56.508197,38.065574
勝個數,14.000000,30.655738,30.295082,19.721311
單次報酬,-0.000802,0.001885,0.009778,0.014875
勝報酬,0.071190,0.061352,0.068280,0.086239
敗報酬,-0.059759,-0.054309,-0.055108,-0.059840
勝率,0.471219,0.494087,0.521392,0.502201
odds,1.191300,1.129700,1.239000,1.441200
凱莉值,0.027400,0.046300,0.135100,0.156800


In [1109]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("營收公告後買進", "使用預估值", "使用預估值，並進行產業分類"))

# '小於-10', '-10到0', '0到10', '10到20','20到30', '30以上'

fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['小於-10'].cumsum(), x=after_yoy.reset_index()['日期'], name='小於-10', marker_color='rgb(102, 166, 30)'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['-10到0'].cumsum(), x=after_yoy.reset_index()['日期'], name='-10到0', marker_color='rgb(179, 222, 105)'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['0到10'].cumsum(), x=after_yoy.reset_index()['日期'], name='0到10', marker_color='rgb(102, 102, 102)'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['10到20'].cumsum(), x=after_yoy.reset_index()['日期'], name='10到20', marker_color='rgb(251, 180, 174)'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['20到30'].cumsum(), x=after_yoy.reset_index()['日期'], name='20到30', marker_color='#E45756'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(y=after_yoy.reset_index()['30以上'].cumsum(), x=after_yoy.reset_index()['日期'], name='30以上', marker_color='rgb(228, 26, 28)'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(y=all_industry.reset_index()['小於-10'].cumsum(), x=all_industry.reset_index()['日期'], name='小於-10', marker_color='rgb(102, 166, 30)'),
    row=2, col=1)
fig.add_trace(
    go.Scatter(y=all_industry.reset_index()['-10到0'].cumsum(), x=all_industry.reset_index()['日期'], name='-10到0', marker_color='rgb(179, 222, 105)'),
    row=2, col=1)
fig.add_trace(
    go.Scatter(y=all_industry.reset_index()['0到10'].cumsum(), x=all_industry.reset_index()['日期'], name='0到10', marker_color='rgb(251, 180, 174)'),
    row=2, col=1)
fig.add_trace(
    go.Scatter(y=all_industry.reset_index()['10以上'].cumsum(), x=all_industry.reset_index()['日期'], name='10以上', marker_color='rgb(228, 26, 28)'),
    row=2, col=1).update_xaxes(showticklabels=False)


fig.add_trace(
    go.Scatter(y=fultered.reset_index()['小於-10'].cumsum(), x=fultered.reset_index()['日期'], name='小於-10', marker_color='rgb(102, 166, 30)'),
    row=3, col=1)
fig.add_trace(
    go.Scatter(y=fultered.reset_index()['-10到0'].cumsum(), x=fultered.reset_index()['日期'], name='-10到0', marker_color='rgb(179, 222, 105)'),
    row=3, col=1)
fig.add_trace(
    go.Scatter(y=fultered.reset_index()['0到10'].cumsum(), x=fultered.reset_index()['日期'], name='0到10', marker_color='rgb(251, 180, 174)'),
    row=3, col=1)
fig.add_trace(
    go.Scatter(y=fultered.reset_index()['10以上'].cumsum(), x=fultered.reset_index()['日期'], name='10以上', marker_color='rgb(228, 26, 28)'),
    row=3, col=1).update_xaxes(showticklabels=True, row=3,col=1)



fig.update_layout(width=800, height=800)

In [1108]:
# px.line(a.pct_change(periods=252).cumsum())
test_result = pd.DataFrame()
test_result['最後報酬'] = a.sum()
test_result['mean'] = a.mean()
test_result['risk'] = a.std()
test_result['sharpe_ratio'] = a.mean() / a.std() * (252 ** 0.5)
test_result
# test.drop_duplicates(['st_code', 'rev_period'], keep='last').groupby(['rev_period', '分類'], as_index=False).agg({'st_code':'count', 'return':'mean'})#.pivot_table(index='分類', columns='rev_period', values='return', aggfunc='mean')


,最後報酬,mean,risk,sharpe_ratio
分類,,,,
小於-10,0.046740,0.000037,0.012931,0.045793
-10到0,0.339484,0.000271,0.011078,0.388242
0到10,0.640938,0.000512,0.011814,0.687351
10以上,0.825225,0.000659,0.013667,0.764951


# 調整產業類別

In [1143]:
predict_data = pd.merge(data.loc[:, ['st_code', 'rev_period', 'declaration_date']], full60, 
                        on=['st_code', 'rev_period'], how='right').dropna(subset='st_code', axis=0)

industry_index = db.drop_duplicates('st_code', keep='first').drop(['st_name', 'rev_period', 'declaration_date','rev','declaration_year','declaration_month'], axis=1)
filter_index = pd.merge(industry_index, st_future.loc[:, ['st_code']].drop_duplicates('st_code', keep='first'), on='st_code', how='inner')

mask = filter_index[(filter_index.new_industry_name != 'M1722 生技醫療') 
                    & (filter_index.new_industry_name != 'M2326 光電業')
                    & (filter_index.new_industry_name != 'M2331 其他電子業') 
                    & (filter_index.new_industry_name != 'M2500 建材營造') 
                    & (filter_index.new_industry_name != 'M2600 航運業') 
                    & (filter_index.new_industry_name != 'M2800 金融業')
                    & (filter_index.new_industry_name != 'M2900 貿易百貨')
                    & (filter_index.new_industry_name != 'M3200 文化創意業')
#                     & (filter_index.new_industry_name != 'M2100 橡膠工業')
#                     & (filter_index.new_industry_name != 'M1800 玻璃陶瓷')
#                     & (filter_index.new_industry_name != 'M3400 電子商務')
#                     & (filter_index.new_industry_name != 'M1900 造紙工業')
                    & (filter_index.minor_industry_name != 'M25A 建設')]


# mask = filter_index[(filter_index.new_industry_name == 'M1722 生技醫療') 
#                     | (filter_index.new_industry_name == 'M2500 建材營造')  
#                     | (filter_index.new_industry_name == 'M2800 金融業')
#                     | (filter_index.new_industry_name == 'M2100 橡膠工業')
#                     | (filter_index.new_industry_name == 'M2900 貿易百貨')
#                     | (filter_index.new_industry_name == 'M3200 文化創意業')
#                     | (filter_index.new_industry_name != 'M2326 光電業')
#                     | (filter_index.minor_industry_name == 'M25A 建設')]

predict_data_f = pd.merge(mask['st_code'], predict_data, on='st_code', how='inner')
mask.new_industry_name.unique()
# filter_index.new_industry_name.unique()

array(['M2324 半導體', 'M2327 通信網路業', 'M1400 紡織纖維', 'M1500 電機機械',
       'M2325 電腦及週邊', 'M2328 電子零組件', 'M9900 其他', 'M1300 塑膠工業',
       'M1600 電器電纜', 'M1721 化學工業', 'M2100 橡膠工業', 'M2329 電子通路業',
       'M1100 水泥工業', 'M1200 食品工業', 'M2200 汽車工業', 'M1800 玻璃陶瓷',
       'M1900 造紙工業', 'M2000 鋼鐵工業', 'M2330 資訊服務業', 'M3400 電子商務'],
      dtype=object)

In [1144]:
pd.set_option('display.max_columns', None)
def get_cluster(data):
#     data['分類'] = pd.qcut(data.yoy_predict60, 8, labels=[1, 2, 3, 4, 5, 6, 7, 8]) #
#     data['分類'] = pd.cut(data.yoy_predict60, bins=[-10000, -0.1, 0, 0.25, 10000000], 
#                         labels=['小於-10', '-10到0', '0到25', '25以上'])#, '25以上'
    data['分類'] = pd.cut(data.yoy_predict60, bins=[-10000, -0.10, 0, 0.1, 10000000], 
                        labels=['小於-10', '-10到0', '0到10', '10以上']) #, '25以上'
    return data

predict_data_f = predict_data_f.groupby('rev_period', as_index=False).apply(get_cluster)
predict_data_f = predict_data_f.dropna(subset='分類')
st_future['rev_period'] = np.where(st_future.day < 11.0, None, st_future['rev_period'])
st_future['rev_period'] = st_future['rev_period'].fillna(method='pad')
# st_future['rev_period'] = st_future['日期'].str[0:7]
# st_future['day'] = st_future['日期'].str[-2:].astype(int)


test_f = pd.merge(st_future[(st_future['近遠月'] == '近月')], predict_data_f,
                on=['st_code', 'rev_period'], how='right').sort_values(['st_code', 'rev_period', '日期'])

test_f = test_f.dropna(subset=['每日結算價'])
# test_f[test_f.st_code=='2330'].drop_duplicates('rev_period', keep='last')

def get_return(data):
    data['return'] = data['每日結算價'].pct_change(periods=1)
    data['cost'] = data.drop_duplicates('rev_period', keep='first')['每日結算價']
    data['cost'] = data['cost'].fillna(method='pad')#.shift(1)
#     data['pnl'] = (data.drop_duplicates('rev_period', keep='last')['每日結算價']/data.drop_duplicates('rev_period', keep='first')['cost'])-1
    return data

test_f = test_f.groupby('st_code', as_index=False).apply(get_return)
test_f[test_f.st_code == '2327'].head(50)


fultered = test_f.pivot_table(values='return', index='日期', columns='分類', aggfunc='mean', fill_value=0)
# # a['bench_mark'] = test.groupby('日期').agg({'return':'mean'})['return']
px.line(fultered.cumsum(), title='過濾產業的結果')

In [1145]:
get_summarize(test_f)
# sharp_ratio(fultered)

分類,小於-10,-10到0,0到10,10以上
交易次數,15.557377,43.967213,42.918033,25.704918
勝個數,8.081967,22.278689,22.950820,13.163934
單次報酬,0.005178,0.002408,0.009772,0.014652
勝報酬,0.078421,0.062379,0.069929,0.090597
敗報酬,-0.061774,-0.054402,-0.055714,-0.061255
勝率,0.485261,0.496516,0.520467,0.493713
odds,1.269500,1.146600,1.255100,1.479000
凱莉值,0.079800,0.057400,0.138400,0.151400


In [1127]:
def get_summarize(data):
    summmariz_f = data.drop_duplicates(['st_code', 'rev_period'], keep='first')

    def get_summarize(data):
        data['單次報酬'] = round(data.cost.pct_change(1), 4)
        data['count'] = 1
        data['交易次數'] = np.where(data['單次報酬'], 1, 0)
        data['勝報酬'] = data.loc[data['單次報酬'] >= 0, '單次報酬']
        data['敗報酬'] = data.loc[data['單次報酬'] < 0, '單次報酬']
        data['勝個數'] = data.loc[data['單次報酬'] >= 0, '單次報酬']
        data['勝率'] = np.where(data['單次報酬'] > 0, 1, 0)
        return data

    summmariz_f = summmariz_f.groupby('st_code', as_index=False).apply(get_summarize)

    dff1 = summmariz_f[summmariz_f.rev_period > '2018/01'].groupby(['rev_period', '分類'], as_index=False).agg(
        {'交易次數':'count','勝個數':'count', '單次報酬':'mean','勝報酬':'mean', '敗報酬':'mean',
         '勝率':'mean'}).groupby('分類').agg('mean')
    dff1['odds'] = round(dff1['勝報酬']/abs(dff1['敗報酬']),4)
    dff1['凱莉值'] = round((dff1['勝率'] - ((1 - dff1['勝率'])/dff1.odds)),4)
    return dff1.transpose()
def sharp_ratio(fultered):
    test_result = pd.DataFrame()
    test_result['最後報酬'] = fultered.sum()
    test_result['mean'] = fultered.mean() * 252
    test_result['risk'] = fultered.std() * (252 ** 0.5)
    test_result['sharpe_ratio'] = fultered.mean() / fultered.std() * (252 ** 0.5)
    return test_result.transpose()

In [1090]:
summmariz_f[(summmariz_f['分類'] == '小於-10') & (summmariz_f.rev_period == '2018/02')]
summmariz_f[summmariz_f.st_code == '3008']
st_future[(st_future.st_code == '3008') & (st_future.rev_period =='2018/01')]

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost
61712,3008,大立光,ZOLF,2018/01/10,2018/01,3748.0769,3810.0000,3722.6923,3762.6923,0.5240,927,1975,近兩月,2018/01,10,2018,01,86.4533
61713,3008,大立光,ZOLFA,2018/01/10,2018/01,3740.0000,3810.0000,3720.0000,3760.0000,0.2667,1768,1975,近月,2018/01,10,2018,01,86.4533
61529,3008,大立光,ZIJFA,2018/01/10,2018/01,3740.0000,3805.0000,3730.0000,3760.0000,0.2667,47,55,近月,2018/01,10,2018,01,86.4533
61528,3008,大立光,ZIJF,2018/01/10,2018/01,3753.4615,3788.8462,3748.8462,3762.6923,0.5240,26,55,近兩月,2018/01,10,2018,01,86.4533
62116,3008,大立光,ZOLF,2018/01/11,2018/01,3793.4615,3857.6923,3728.0769,3738.0769,-0.6542,732,1814,近兩月,2018/01,11,2018,01,3738.0769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69987,3008,大立光,ZIJFA,2018/02/08,2018/02,3695.0000,3740.0000,3680.0000,3720.0000,2.1978,16,19,近月,2018/01,8,2018,02,84.6000
70571,3008,大立光,ZOLFA,2018/02/09,2018/02,3590.0000,3630.0000,3475.0000,3595.0000,-3.3602,1192,1472,近月,2018/01,9,2018,02,84.6000
70570,3008,大立光,ZOLF,2018/02/09,2018/02,3562.0000,3626.5000,3478.5000,3591.5000,-3.4543,551,1472,近兩月,2018/01,9,2018,02,84.6000
70389,3008,大立光,ZIJFA,2018/02/09,2018/02,3550.0000,3605.0000,3550.0000,3595.0000,-3.3602,31,31,近月,2018/01,9,2018,02,84.6000


In [1067]:
# px.line(a.pct_change(periods=252).cumsum())
def sharp_ratio(fultered):
    test_result = pd.DataFrame()
    test_result['最後報酬'] = fultered.sum()
    test_result['mean'] = fultered.mean() * 252
    test_result['risk'] = fultered.std() * (252 ** 0.5)
    test_result['sharpe_ratio'] = fultered.mean() / fultered.std() * (252 ** 0.5)
    return test_result.transpose()
sharp_ratio(fultered)
get_summarize(test)
test

,st_code,st_name,代號,日期,到期月,開盤價,最高價,最低價,每日結算價,報酬率,成交張數(量),成交量(合計),近遠月,rev_period,day,year,month,cost,declaration_date,new_industry_name,rev_predict60,mom_predict60,yoy_predict60,分類,return
137326,1101,台泥,ZDFFA,2018/01/12,2018/01,38.50,38.50,37.45,37.45,-0.1333,97.0,170.0,近月,2018/01,12.0,2018,01,37.45,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,NaN
137327,1101,台泥,ZDFFA,2018/01/15,2018/01,37.65,37.85,37.65,37.85,1.0681,106.0,238.0,近月,2018/01,15.0,2018,01,37.45,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.010681
137328,1101,台泥,ZDFFA,2018/01/16,2018/01,37.90,38.10,37.90,38.00,0.3963,36.0,40.0,近月,2018/01,16.0,2018,01,37.45,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.003963
137329,1101,台泥,ZDFFA,2018/01/17,2018/01,38.05,38.55,38.05,38.35,0.9211,270.0,579.0,近月,2018/01,17.0,2018,01,37.45,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,0.009211
137330,1101,台泥,ZDFFA,2018/01/18,2018/02,38.50,38.65,37.90,37.90,-1.1734,161.0,270.0,近月,2018/01,18.0,2018,01,37.45,2018/02/12,M1100 水泥工業,7049.0,-0.358491,0.1496,10以上,-0.011734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176722,9939,宏全,ZKOFA,2023/02/06,2023/02,91.50,91.50,90.80,91.10,-0.4372,10.0,11.0,近月,2023/01,6.0,2023,02,86.50,NaN,NaN,1991.0,0.119231,0.0142,0到10,-0.004372
176723,9939,宏全,ZKOFA,2023/02/07,2023/02,91.10,91.70,91.10,91.70,0.6586,96.0,186.0,近月,2023/01,7.0,2023,02,86.50,NaN,NaN,1991.0,0.119231,0.0142,0到10,0.006586
176724,9939,宏全,ZKOFA,2023/02/08,2023/02,91.60,95.10,91.60,94.70,3.2715,70.0,109.0,近月,2023/01,8.0,2023,02,86.50,NaN,NaN,1991.0,0.119231,0.0142,0到10,0.032715
176725,9939,宏全,ZKOFA,2023/02/09,2023/02,95.20,96.50,93.80,93.80,-0.9504,33.0,38.0,近月,2023/01,9.0,2023,02,86.50,NaN,NaN,1991.0,0.119231,0.0142,0到10,-0.009504


In [571]:
px.line(test.groupby('yoy_predict60').agg({'return':'mean'}).fillna(0).cumsum())

# 用真實的營收數據，公布後再進場
1. 每月公佈完後調整持倉部位，用收盤價進場
2. 2018/01 的數據，2018/02/01 - 10會公布，故持有期間為 02/13 - 03/13

In [1004]:
predict_data = pd.merge(data.loc[:, ['st_code', 'rev_period', 'declaration_date', 'new_industry_name']], full60, 
                        on=['st_code', 'rev_period'], how='right').dropna(subset='st_code', axis=0)

predict_data_real = pd.merge(predict_data, data.loc[:, ['st_code', 'rev_period', 'rev', 'mom', 'yoy']], on=['st_code', 'rev_period'], how='left')

def reset_rev_period(data):
    data['rev_period'] = data['rev_period'].shift(-1)
    return data

def get_cluster(data):
#     data['分類'] = pd.qcut(data.yoy_predict60, 4, labels=[1, 2, 3, 4]) #
    data['分類'] = pd.cut(data.yoy, bins=[-10000, -0.10, 0, 0.10, 0.20, 0.3, 10000000], 
                        labels=['小於-10', '-10到0', '0到10', '10到20','20到30', '30以上']) #, '25以上'
    return data


st_future['rev_period'] = st_future['日期'].str[0:7]
st_future['rev_period'] = np.where(st_future.day > 10.0, None, st_future['rev_period'])
st_future['rev_period'] = st_future['rev_period'].fillna(method='bfill')
# st_future[st_future.st_code == '1101'].head(50)
predict_data_real = predict_data_real.groupby('rev_period', as_index=False).apply(get_cluster)
predict_data_real = pd.merge(st_future[(st_future['近遠月'] == '近月') & (st_future.st_code.isin(predict_data_real.st_code) == True)].drop('rev_period', axis=1), predict_data_real.rename({'declaration_date':'日期'}, axis=1).drop(['rev_predict60','mom_predict60','yoy_predict60'], axis=1), on=['st_code', '日期'], how='left').sort_values(['st_code','日期'])
predict_data_real = predict_data_real.fillna(method='pad').dropna(subset='rev_period')

test_real = predict_data_real.groupby('st_code', as_index=False).apply(get_return)
test_real.drop_duplicates(['st_code', 'rev_period'], keep='first')['return'] = 0
after_yoy = test_real.pivot_table(values='return', index='日期', columns='分類', aggfunc='mean', fill_value=0)
# a['bench_mark'] = test_real.groupby('日期').agg({'return':'mean'})['return']
# px.line(after_yoy.cumsum())

In [1054]:
get_summarize(test_real)
# sharp_ratio(after_yoy)


分類,小於-10,-10到0,0到10,10到20,20到30,30以上
交易次數,44.283333,27.683333,32.850000,25.666667,16.566667,31.900000
勝個數,19.366667,13.833333,16.500000,13.600000,9.416667,17.516667
單次報酬,-0.006526,0.003848,0.005433,0.010124,0.030084,0.029986
勝報酬,0.066681,0.062298,0.066199,0.069187,0.091359,0.112798
敗報酬,-0.060925,-0.056915,-0.057333,-0.059220,-0.070236,-0.067366
勝率,0.428224,0.492657,0.484012,0.520631,0.554672,0.539795
odds,1.094500,1.094600,1.154600,1.168300,1.300700,1.674400
凱莉值,-0.094200,0.029200,0.037100,0.110300,0.212300,0.264900


In [ ]:
# test_real# # We are going to use a trailing 252 trading day window
# window = 252

# # Calculate the max drawdown in the past window days for each day in the series.
# # Use min_periods=1 if you want to let the first 252 days data have an expanding window
# Roll_Max = a.iloc[:, 1:5].rolling(window, min_periods=1).max()
# Daily_Drawdown = a.iloc[:, 1:5]/Roll_Max - 1.0

# # Next we calculate the minimum (negative) daily drawdown in that window.
# # Again, use min_periods=1 if you want to allow the expanding window
# Max_Daily_Drawdown = Daily_Drawdown.rolling(window, min_periods=1).min()

In [150]:
result.tail(10)
get_predict(data[data.st_code == '9945'].tail(24))
# data[data.st_code == '9945'].head(24)

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,month,mom,yoy
12,9945,潤泰新,* 2023/02,2022/03/10,1415.0,2022,3,M9900 其他,M25A 建設,2023,02,-0.153430,-0.0474
13,9945,潤泰新,* 2023/03,2022/04/11,2042.0,2022,4,M9900 其他,M25A 建設,2023,03,0.443110,-0.1151
14,9945,潤泰新,* 2023/04,2022/05/10,2393.0,2022,5,M9900 其他,M25A 建設,2023,04,0.171890,-0.0513
15,9945,潤泰新,* 2023/05,2022/06/10,2213.0,2022,6,M9900 其他,M25A 建設,2023,05,-0.075219,0.2731
16,9945,潤泰新,* 2023/06,2022/07/11,2747.0,2022,7,M9900 其他,M25A 建設,2023,06,0.241301,-0.1884
17,9945,潤泰新,* 2023/07,2022/08/10,3057.0,2022,8,M9900 其他,M25A 建設,2023,07,0.112850,0.0738
18,9945,潤泰新,* 2023/08,2022/09/12,3175.0,2022,9,M9900 其他,M25A 建設,2023,08,0.038600,0.3663
19,9945,潤泰新,* 2023/09,2022/10/11,2405.0,2022,10,M9900 其他,M25A 建設,2023,09,-0.242520,-0.0848
20,9945,潤泰新,* 2023/10,2022/11/10,2290.0,2022,11,M9900 其他,M25A 建設,2023,10,-0.047817,-0.0298
21,9945,潤泰新,* 2023/11,2022/12/09,2445.0,2022,12,M9900 其他,M25A 建設,2023,11,0.067686,-0.1100


In [5]:
a = db.groupby('st_code', as_index=False).apply(get_st_mom_yoy).reset_index(drop=True)
predict_df = a.groupby('st_code', as_index=False).apply(predict).reset_index(drop=True)


# johnny 合併歷史跟預測資料，計算最新一期的表現是成長 or 衰退
johnny = pd.concat([a, predict_df], axis=0).sort_values(['st_code', 'Year', 'month'])



# 合併後計算檔案
concat = johnny.groupby('st_code', as_index=False).apply(growth).reset_index(drop=True)
concat

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
0,1101,台泥,2013/01,2013/02/18,9134465.0,2013,2,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
1,1101,台泥,2013/02,2013/03/08,5540346.0,2013,3,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
2,1101,台泥,2013/03,2013/04/10,9457971.0,2013,4,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
3,1101,台泥,2013/04,2013/05/10,9919269.0,2013,5,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
4,1101,台泥,2013/05,2013/06/10,9543782.0,2013,6,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212899,9962,有益,* 2023/09,2022/10/06,292786.0,2022,10,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,0.1079,0,1,0,0.0,0,3.0,1,2.0,0
212900,9962,有益,* 2023/10,2022/11/04,269637.0,2022,11,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,-0.1229,0,0,1,0.0,0,2.0,0,3.0,1
212901,9962,有益,* 2023/11,2022/12/07,336802.0,2022,12,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,0.0912,0,1,0,0.0,0,3.0,1,2.0,0
212902,9962,有益,* 2023/12,2023/01/09,291156.0,2023,1,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,-0.2451,0,0,1,0.0,0,2.0,0,3.0,1


In [6]:
taifex_product = pd.read_html('https://www.taifex.com.tw/cht/5/stockMargining')[0].drop(['序號', '股票期貨標的證券'], axis=1)
taifex_product = taifex_product.rename({'股票期貨標的證券代號':'代號', '股票期貨  中文簡稱':'名稱'}, axis=1)
taifex_product['名稱'] = taifex_product['名稱'].str[:-2]
taifex_product
dividend = pd.read_excel('15_23股利.xlsx').rename({'盈餘分派_年度':'年/月'}, axis=1)

taifex_dividend = dividend[dividend['代號'].isin(taifex_product['代號']) == True]
taifex_profit = profit[profit['代號'].astype(int).isin(taifex_product['代號']) == True]
taifex_db = db[db.st_code.astype(int).isin(taifex_product['代號']) == True]
taifex_concat = concat[concat.st_code.astype(int).isin(taifex_product['代號']) == True]
taifex_concat

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
0,1101,台泥,2013/01,2013/02/18,9134465.0,2013,2,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
1,1101,台泥,2013/02,2013/03/08,5540346.0,2013,3,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
2,1101,台泥,2013/03,2013/04/10,9457971.0,2013,4,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
3,1101,台泥,2013/04,2013/05/10,9919269.0,2013,5,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
4,1101,台泥,2013/05,2013/06/10,9543782.0,2013,6,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211838,9945,潤泰新,* 2023/09,2022/10/11,2666937.0,2022,10,M9900 其他,M25A 建設,2023,...,0.0149,0,0,1,0.0,0,3.0,0,2.0,1
211839,9945,潤泰新,* 2023/10,2022/11/10,2309969.0,2022,11,M9900 其他,M25A 建設,2023,...,-0.0214,0,0,1,0.0,0,2.0,0,3.0,2
211840,9945,潤泰新,* 2023/11,2022/12/09,2658087.0,2022,12,M9900 其他,M25A 建設,2023,...,-0.0325,0,0,1,0.0,0,2.0,0,3.0,3
211841,9945,潤泰新,* 2023/12,2023/01/10,3392675.0,2023,1,M9900 其他,M25A 建設,2023,...,-0.3471,0,0,1,0.0,0,1.0,0,4.0,4


In [9]:
taifex_concat[taifex_concat.rev_period == '* 2023/02']

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
121,1101,台泥,* 2023/02,2022/03/10,4562615.0,2022,3,M1100 水泥工業,M11A 水泥製造,2023,...,-0.1638,0,0,1,0.0,0,2.0,0,3.0,2
254,1102,亞泥,* 2023/02,2022/03/10,3220684.0,2022,3,M1100 水泥工業,M11A 水泥製造,2023,...,-0.2146,0,1,0,0.0,0,2.0,1,3.0,0
1318,1210,大成,* 2023/02,2022/03/09,6800021.0,2022,3,M1200 食品工業,M12B 油脂飼料,2023,...,-0.0953,0,0,1,0.0,0,1.0,0,4.0,4
1716,1216,統一,* 2023/02,2022/03/10,33064258.0,2022,3,M1200 食品工業,M12C 飲料乳品,2023,...,-0.0307,0,1,0,0.0,0,2.0,1,3.0,0
4276,1301,台塑,* 2023/02,2022/03/08,14982217.0,2022,3,M1300 塑膠工業,M13A1 泛用塑膠,2023,...,-0.2724,0,1,0,0.0,0,2.0,1,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207594,9904,寶成,* 2023/02,2022/03/10,18053812.0,2022,3,M9900 其他,M99K 鞋及旅行箱,2023,...,-0.0076,0,1,0,0.0,0,2.0,2,3.0,0
208645,9914,美利達,* 2023/02,2022/03/10,2580094.0,2022,3,M9900 其他,M99Z1 自行健行車,2023,...,0.1232,0,1,0,0.0,0,2.0,1,3.0,0
210902,9938,百和,* 2023/02,2022/03/07,1034889.0,2022,3,M9900 其他,M14Z 其他紡織,2023,...,-0.0666,0,1,0,0.0,0,2.0,1,3.0,0
211035,9939,宏全,* 2023/02,2022/03/10,1538779.0,2022,3,M9900 其他,M13B 塑膠製品,2023,...,0.0063,0,0,1,0.0,0,1.0,0,4.0,3


In [29]:
johnny[johnny.st_code == '8069'].loc[:,['st_code', 'st_name', 'rev_period', 'rev', 'yoy', 'mom']].tail(37)

,st_code,st_name,rev_period,rev,yoy,mom
172243,8069,元太,2021/01,1559034.0,0.5963,0.219500
172244,8069,元太,2021/02,1356295.0,1.0281,-0.130000
172245,8069,元太,2021/03,1520435.0,0.1909,0.121000
172246,8069,元太,2021/04,1428276.0,-0.0534,-0.060600
172247,8069,元太,2021/05,1237551.0,0.0462,-0.133500
172248,8069,元太,2021/06,1384590.0,0.3325,0.118800
172249,8069,元太,2021/07,1664313.0,0.1887,0.202000
172250,8069,元太,2021/08,861190.0,-0.4323,-0.482600
172251,8069,元太,2021/09,1871257.0,0.2192,1.172900
172252,8069,元太,2021/10,2206347.0,0.3819,0.179100
